## Scenario 2: A cross-functional team with one data scientist working on an ML model


MLflow setup:
- tracking server: yes, local server
- backend store: sqlite database
- artifacts store: local filesystem

The experiments can be explored locally by accessing the local tracking server.

To run this example you need to launch the mlflow server locally by running the following command in your terminal:

`mlflow server --backend-store-uri sqlite:///backend.db`

In [54]:
import mlflow
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score
import json

In [2]:
mlflow.set_tracking_uri("http://127.0.0.1:5000")

In [ ]:
print(f"tracking URI: '{mlflow.get_tracking_uri()}'")

In [7]:
mlflow.get_experiment(experiment_id=0)

<Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1719157481632, experiment_id='0', last_update_time=1719157481632, lifecycle_stage='active', name='Default', tags={}>

In [8]:
mlflow.set_experiment("my-experiment-1")

with mlflow.start_run():

    X, y = load_iris(return_X_y=True)

    params = {"C": 0.1, "random_state": 42}
    mlflow.log_params(params)

    lr = LogisticRegression(**params).fit(X, y)
    y_pred = lr.predict(X)
    mlflow.log_metric("accuracy", accuracy_score(y, y_pred))

    mlflow.sklearn.log_model(lr, artifact_path="models")
    print(f"default artifacts URI: '{mlflow.get_artifact_uri()}'")

2024/06/23 15:50:00 INFO mlflow.tracking.fluent: Experiment with name 'my-experiment-1' does not exist. Creating a new experiment.


default artifacts URI: 'mlflow-artifacts:/1/5dc9b8a1bcab400182dab6ec7da10d67/artifacts'


### Interacting with the model registry

In [10]:
from mlflow.tracking import MlflowClient
client = MlflowClient("http://127.0.0.1:5000")

In [14]:
client.search_registered_models()

[]

In [60]:
run_data = client.search_runs(experiment_ids='1')[0].data
run_data_dict = run_data.to_dictionary()
list = json.loads(run_data_dict['tags']['mlflow.log-model.history'])
run_id = list[0]['run_id']

In [61]:
mlflow.register_model(
    model_uri=f"runs:/{run_id}/models",
    name='iris-classifier'
)

Successfully registered model 'iris-classifier'.
2024/06/23 16:13:33 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: iris-classifier, version 1
Created version '1' of model 'iris-classifier'.


<ModelVersion: aliases=[], creation_timestamp=1719159213044, current_stage='None', description='', last_updated_timestamp=1719159213044, name='iris-classifier', run_id='5dc9b8a1bcab400182dab6ec7da10d67', run_link='', source='mlflow-artifacts:/1/5dc9b8a1bcab400182dab6ec7da10d67/artifacts/models', status='READY', status_message='', tags={}, user_id='', version='1'>